
 # Cloud-Native Monitoring with EBPF

gergely.szabo@origoss.com

# About Myself
* more than 15 years in the industry
* research, development, system architect, etc...
* currently at Origoss Solutions
  * Cloud Native
  * Kubernetes
  * Prometheus

# Agenda
* BPF
* Linux Kernel Tracing
* EBPF
* Prometheus Exporter

# BPF

# Packet Filtering Problem
  
![network filtering](filter.svg "Network filtering")

# Filtering Requirements

* Efficient
* Flexible filter rules
* Safe

# Where to Filter?

![Where to filter?](filter2.svg "Where to Filter?")

# BPF

## Steven McCanne and Van Jacobson:

## The BSD Packet Filter: A New Architecture for User-level Packet Capture, 1992

http://www.tcpdump.org/papers/bpf-usenix93.pdf

# BPF Architecture

![BPF Architecture](bpf_paper_fig1.png "BPF Architecture")

# Capturing without Filtering

In [ ]:
%%bash
tcpdump -nc 4

# Simple Filtering Rule

In [ ]:
%%bash
tcpdump -nc 4 tcp and port 80

# Complex Rule

To print all IPv4 HTTP packets to and from port 80, i.e. print only packets that contain data, not, for example, SYN and FIN packets and ACK-only packets.

In [ ]:
%%bash
tcpdump -nc 4 'tcp port 80 and (((ip[2:2] - ((ip[0]&0xf)<<2)) - ((tcp[12]&0xf0)>>2)) != 0)'

# How Does This Work?

![BPF Architecture](bpf_paper_fig1.png "BPF Architecture")

# BPF VM Instruction Set

![BPF Instructions](bpf_instructions.png "BPF Instructions")

# Simple Filtering Rule

In [ ]:
%%bash
tcpdump -d tcp and port 80

# Complex Rule

In [ ]:
%%bash
tcpdump -d 'tcp port 80 and (((ip[2:2] - ((ip[0]&0xf)<<2)) - ((tcp[12]&0xf0)>>2)) != 0)'

# Linux Kernel Tracepoints

* A tracepoint placed in code provides a hook to call a function (probe) that you can provide at runtime. 
* A tracepoint can be "on" pr "off" 
 * When a tracepoint is "on", the function you provide is called each time the tracepoint is executed
* They can be used for tracing and performance accounting.


In [ ]:
%%bash
perf list